In [7]:
'''
The baseline model
from  http://cdn2.jianshu.io/p/ed4b49c2d16b
https://blog.floydhub.com/colorizing-b&w-photos-with-neural-networks/
the input images are 256x256
'''
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
from skimage.transform import resize
import numpy as np
import os
import random
import tensorflow as tf



In [8]:
# Get images
#resize(image, output_shape)
X =[]
#./Mask_RCNN/images
for filename in os.listdir('./images/small/Train'):
    X.append(img_to_array(load_img('./images/small/train/'+filename,target_size=(256,256))))

X = np.array(X)
print(X.shape)
print(X[0].shape)
#X = np.array(X,dtype=np.float32)

# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain


(10, 256, 256, 3)
(256, 256, 3)


In [9]:
#design the neural model
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
# finish model
model.compile(optimizer='rmsprop', loss='mse')



In [15]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)


# Generate training data
#change batch to 50-100
#steps_per_epoch=totoal number/batch
batch_size = 10
#make sure the images will not be the same through iterations
def image_a_b_gen(batch_size):
    #datagen.flow(Xtrain, batch_size=batch_size)
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model      
tensorboard = TensorBoard(log_dir="log/baseline/output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=1, steps_per_epoch=10)


Epoch 1/1
10/10 [==============================] - 123s 12s/step - loss: 0.9165


In [16]:
# Save model
model_json = model.to_json()
with open("log/baseline/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("log/baseline/model.h5")


In [17]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))


1/1 [==============================] - 0s 436ms/step
1.0009868145


In [18]:
#still train data
color_me = []
#./Mask_RCNN/images/
for filename in os.listdir('./images/small/Validate'):
    color_me.append(img_to_array(load_img('./images/small/Validate/'+filename,target_size=(256,256))))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("log/baseline/output/img1_"+str(i)+".png", lab2rgb(cur))


/Users/Melanie/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 52083 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/Users/Melanie/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/Melanie/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 63457 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/Users/Melanie/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 53386 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/Users/Melanie/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/color/colorconv